In [ ]:
import json

# 打开并读取JSON文件
with open('dureadqg/train.json', 'r') as file:
    data = json.load(file)

# 输出读取到的数据
print(data)

In [1]:
import json

# 定义一个空列表用于存储所有的字典
train_list = []
test_list = []
# 打开JSON Lines文件
with open('dureadqg/train.json', 'r') as file:
    # 逐行读取
    for line in file:
        # 解析每一行
        data_dict = json.loads(line)
        # 将解析后的字典添加到列表中
        train_list.append(data_dict)
# 打开JSON Lines文件
with open('dureadqg/dev.json', 'r') as file:
    # 逐行读取
    for line in file:
        # 解析每一行
        data_dict = json.loads(line)
        # 将解析后的字典添加到列表中
        test_list.append(data_dict)


In [2]:
print(len(train_list), len(test_list))

14520 984


In [9]:
import numpy as np
con, que = [], []
for i in range(len(train_list)):
    con.append(len(train_list[i]['context']))
    que.append(len(train_list[i]['question']))

In [13]:
np.mean(con), np.max(que)

(282.2969696969697, 42)

In [ ]:
!pip install datasets transformers rouge-score nltk

In [14]:
from datasets import *
train_dataset = Dataset.from_list(train_list)
test_dataset = Dataset.from_list(test_list)
print(train_dataset, test_dataset)

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 14520
}) Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 984
})


In [1]:
!git lfs install

Updated Git hooks.
Git LFS initialized.


In [2]:
!git clone https://huggingface.co/uer/t5-base-chinese-cluecorpussmall

正克隆到 't5-base-chinese-cluecorpussmall'...
fatal: 无法访问 'https://huggingface.co/uer/t5-base-chinese-cluecorpussmall/'：Failed to connect to huggingface.co port 443: 连接超时


In [3]:
from modelscope.hub.api import HubApi

YOUR_ACCESS_TOKEN = '49834fc2-f98f-4ac3-8efc-01ce1764baf8'

api = HubApi()
api.login(YOUR_ACCESS_TOKEN)
api.push_model(
    model_id="zlyzly/t5-base-chinese-cluecorpussmall", 
    model_dir="t5-base-chinese-cluecorpussmall" # 本地模型目录，要求目录中必须包含configuration.json
)

2024-07-24 10:21:54,294 - modelscope - INFO - [master ee39e78] 'upload model'
 5 files changed, 21385 insertions(+)
 create mode 100644 README.md
 create mode 100644 config.json
 create mode 100644 special_tokens_map.json
 create mode 100644 tokenizer_config.json
 create mode 100644 vocab.txt



In [15]:
from modelscope import HubApi
from modelscope import snapshot_download

api=HubApi()
api.login('49834fc2-f98f-4ac3-8efc-01ce1764baf8')

# download your model, the model_path is downloaded model path.
model_path =snapshot_download(model_id='zlyzly/t5-base-chinese-cluecorpussmall')


2024-07-24 15:13:04,469 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2024-07-24 15:13:04,472 - modelscope - INFO - TensorFlow version 2.13.0 Found.
2024-07-24 15:13:04,472 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-07-24 15:13:04,499 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 e096d041fdba41d8d7d7331b3f469288 and a total number of 945 components indexed
Downloading: 100%|██████████| 460/460 [00:00<00:00, 65.2kB/s]
Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 4.03kB/s]
Downloading: 100%|█████████▉| 818M/818M [00:03<00:00, 217MB/s] 
Downloading: 100%|██████████| 6.32k/6.32k [00:00<00:00, 2.83MB/s]
Downloading: 100%|██████████| 263/263 [00:00<00:00, 153kB/s]
Downloading: 100%|██████████| 108k/108k [00:00<00:00, 5.94MB/s]


In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [19]:
print(train_dataset, test_dataset)

Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 14520
}) Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 984
})


In [29]:
len(train_dataset['answer']), len(train_dataset['context'])
q_c_tent = ["问题内容: \n" + e['question'] + "\n文本内容: \n" + e['context'] for e in train_dataset]
answer = [e['answer']  for e in train_dataset]

In [32]:
print(len(q_c_tent), len(answer))
inputs = tokenizer(q_c_tent, max_length=512, truncation=True)
labels = tokenizer(answer, max_length=64, truncation=True)

14520 14520


In [36]:
print(len(labels['input_ids']))

14520


In [37]:
inputs["labels"] = labels["input_ids"]

In [41]:
print(len(inputs['labels']))

14520


In [48]:
train_dataset

Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 14520
})

In [56]:
def process_func(exmaples):
    # print(type(exmaples))
    # q_c_tent = ["问题内容: \n" + e['question'] + "\n文本内容: \n" + e['context'] for e in exmaples]
    qc = ["问题内容: \n" + q + "\n文本内容: \n" + c for q, c in zip(exmaples['question'], exmaples['context'])]
    # q = ["问题内容: \n" + e for e in exmaples['question']]
    # print(q_c_tent, answer)
    inputs = tokenizer(qc, max_length=512, truncation=True)
    labels = tokenizer(text_target=exmaples["answer"], max_length=64, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

train_tokenized_ds = train_dataset.map(process_func, batched=True)
test_tokenized_ds = test_dataset.map(process_func, batched=True)

In [57]:
test_tokenized_ds

Dataset({
    features: ['context', 'answer', 'question', 'id', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 984
})

In [16]:
model_path

'/mnt/workspace/.cache/modelscope/zlyzly/t5-base-chinese-cluecorpussmall'

In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
model